In [1]:
import random
import numpy as np
import torch
import os
import pathlib
import pickle
from gurobipy import *
from rsome import ro
from rsome import grb_solver as grb
import rsome as rso
from rsome import cpt_solver as cpt
import pandas as pd

In [2]:
# data_generation_process = "SPO_Data_Generation"
data_generation_process = "DDR_Data_Generation"

# Parameters

In [ ]:
# import pyepo
# generate data
grid = (5,5) # grid size
num_train = 100 # number of training data
num_feat = 5 # size of feature
num_test = 10000
deg = 1.0 # polynomial degree
e = 1.0 # scale of normal std or the range of uniform. For the error term

lower = 0 # coef lower bound
upper = 1 # coef upper bound
p = num_feat # num of features
d = (grid[0] - 1) * (grid[0] - 1) * 2 + 2 * (grid[0] - 1) # num of arcs
num_nodes = grid[0]*grid[0]
alpha = e # scale of normal std or the range of uniform. For the error term
mis = deg # model misspecification
coef_seed = 1

x_dist = 'uniform'
e_dist = 'normal'
x_low = -2
x_up = 2
x_mean = 2
x_var = 2
bump = 7

In [ ]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
grandparent_directory = os.path.dirname(parent_directory)
DataPath = os.path.dirname(grandparent_directory) + '/Data/Shortest_Path_'+str(grid[0])+'by'+str(grid[1])+'_grid_0404_' + data_generation_process + "/"
pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)
print("grandparent_directory:", grandparent_directory)
print("DataPath:", DataPath)
DataPath = DataPath + "data_size="+str(num_train)+"_deg="+str(deg)+"_e="+str(e)+"_d="+str(d)+"_x_dist="+x_dist+"_coef_seed="+str(coef_seed)+"_same_W/"
pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)

grandparent_directory: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Code_MacBook
DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_2by2_grid_0404_DDR_Data_Generation/


# Generate Data

In [5]:
def Prepare_Data(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump):
# #  ****** Coef generation *********
    from Data import data_generation
    data_gen = data_generation()
    # W_star = data_gen.generate_truth(DataPath,lower, upper, p, d, coef_seed,data_generation_process) 
    # print("W_star = ",W_star[0,:])
    np.random.seed(coef_seed)
    x_test_all = {}; c_test_all = {}; x_train_all = {}; c_train_all = {}; W_star_all = {}
    for iter in iteration_all:
        DataPath_iter = DataPath +"iter="+str(iter)+"/"
        pathlib.Path(DataPath_iter).mkdir(parents=True, exist_ok=True)
        W_star = data_gen.generate_truth(DataPath_iter,lower, upper, p, d, iter,data_generation_process) 
        # #  ****** Data generation *********
        x_test_all[iter], c_test_all[iter], x_train_all[iter], c_train_all[iter], W_star_all[iter] = data_gen.generate_samples(iter,DataPath_iter,p, d, num_test, num_train, alpha, W_star, mis, num_test, 
                                data_generation_process, x_dist, e_dist, x_low, x_up, x_mean, x_var, bump) 
        # print()
    return x_test_all, c_test_all, x_train_all, c_train_all, W_star_all

# EPO(SPO+,...)

In [6]:
# from PYEPO import PyEPO_Method
# epo_runner = PyEPO_Method()
def Implement_EPO(DataPath,seed_all,batch_size,num_epochs,method_names,x_train_all,c_train_all,x_test_all,c_test_all,arcs,epo_runner):
    cost_EPO = {}
    for seed in seed_all:
        DataPath_seed = DataPath +"Seed="+str(seed)+"/"
        pathlib.Path(DataPath_seed).mkdir(parents=True, exist_ok=True)
        # print("*** seed = ",seed,": Run EPO ******")
        cost_EPO[seed] = epo_runner.run(method_names,DataPath_seed,batch_size,num_feat,grid,num_epochs,\
                                        x_train_all[seed],c_train_all[seed],x_test_all[seed],c_test_all[seed],arcs)
    return cost_EPO

# Obtain DDR estimation

### define network

In [7]:
def _getArcs(grid):
    arcs = []
    for i in range(grid[0]):
        # edges on rows
        for j in range(grid[1] - 1):
            v = i * grid[1] + j
            arcs.append((v, v + 1))
        # edges in columns
        if i == grid[0] - 1:
            continue
        for j in range(grid[1]):
            v = i * grid[1] + j
            arcs.append((v, v + grid[1]))

    arc_index_mapping = {}
    for i in range(len(arcs)):
        arc = arcs[i]
        arc_index_mapping[arc] = i

    return arcs,arc_index_mapping

In [8]:
def solve_DDR(arcs,lamb,mu_fixed,num_nodes,x_train,c_train):
    
    N,p = x_train.shape
    N,d = c_train.shape

    # DDR
    m = Model("ddr")
    #m.setParam("DualReductions",0)
    m.setParam('OutputFlag', 0)

    W_ind = tuplelist( [(i,j) for i in range(d) for j in range(p)] )
    w0_ind = tuplelist( [i for i in range(d)])

    W_ddr = m.addVars(W_ind, lb=-GRB.INFINITY,name = "W" )
    w0_ddr = m.addVars(w0_ind, lb=-GRB.INFINITY,name = "W0" )
    alpha = m.addVars(N,num_nodes,lb=-GRB.INFINITY,name="alpha")
    expr_obj = 0
    err = []
    for n in range(N):
        cost_true_tem = c_train[n]
        expr_obj = expr_obj + alpha[n,num_nodes-1] - alpha[n,0]
        for ind in range(len(arcs)):
            cost_pred_tem = quicksum([W_ddr[ind,j] * x_train[n,j] for j in range(p)]) + w0_ddr[ind]
            err.append(cost_true_tem[ind] - cost_pred_tem)
            e = arcs[ind]
            j = e[1]
            i = e[0]
            # print("j = ",j,", i = ",i, ", e = ",e)
            m.addConstr(alpha[n,j] - alpha[n,i] >= -mu_fixed*cost_true_tem[ind] - (1-mu_fixed)*cost_pred_tem)

    m.setObjective(quicksum([err[k] * err[k] for k in range(len(err))])/N + lamb*(expr_obj)/N, GRB.MINIMIZE)
    m.optimize()
    
    W_DDR_rst = m.getAttr('x', W_ddr)
    w0_DDR_rst = m.getAttr('x', w0_ddr)
    W_ddr_val = []
    for i in range(d):
        W_ddr_val.append([W_DDR_rst[(i,j)] for j in range(p)])
    w0_ddr_val = [w0_DDR_rst[i] for i in range(d)]

    alpha_rst = m.getAttr('x', alpha)
    return w0_ddr_val,W_ddr_val,alpha_rst,m.ObjVal

# Out-of-sample performance

### Generate data

In [9]:
arcs,arc_index_mapping = _getArcs(grid)
num_arcs = len(arcs)
iteration_all = np.arange(0,10)
# obtain data
x_test_all, c_test_all, x_train_all, c_train_all, W_star_all = Prepare_Data(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump)

### EPO performance

In [10]:
# # EPO, including SPO, PG, LTR
# batch_size = 20
# num_epochs = 100
# method_names = ["spo+","pg","ltr"]
# method_names = ["spo+"]
# from PYEPO import PyEPO_Method
# epo_runner = PyEPO_Method()

# cost_EPO_all = Implement_EPO(DataPath,[1],batch_size,num_epochs,method_names,x_train_all,c_train_all,x_test_all,c_test_all,arcs,epo_runner)
# print("SPO+Cost = ",np.nanmean(cost_EPO_all[1]["SPO"]))

## Oracle performance

In [11]:
from Peformance import performance_evaluation
perfs = performance_evaluation()
cost_Oracle_para_all = {}; cost_Oracle_para_avg = {}
cost_Oracle_realization_all = {}; cost_Oracle_realization_avg = {}

for iter in iteration_all:
    if data_generation_process == "SPO_Data_Generation":
        cost_dem = (W_star_all[iter] @ x_test_all[iter].T)/np.sqrt(num_feat) + 3
        oracle_cost_pred = (cost_dem ** mis + 1).T
    if data_generation_process == "DDR_Data_Generation":
        cost_dem = (W_star_all[iter] @ x_test_all[iter].T) + bump
        oracle_cost_pred = (cost_dem ** mis).T
        cost_Oracle_para_all[iter] = perfs.compute_Oracel_Cost(arcs, grid,oracle_cost_pred,c_test_all[iter])
        cost_Oracle_para_avg[iter] = np.nanmean(cost_Oracle_para_all[iter])
        
        cost_Oracle_realization_all[iter] = perfs.compute_Oracel_Cost(arcs, grid,c_test_all[iter],c_test_all[iter])
        cost_Oracle_realization_avg[iter] = np.nanmean(cost_Oracle_realization_all[iter])
    # cost_Oracle_all[iter] = perfs.compute_Cost_with_Prediction(arcs,np.ones(num_arcs)*bump, W_star_all[iter],grid,c_test_all[iter],x_test_all[iter])

Set parameter Username
Academic license - for non-commercial use only - expires 2026-03-13


### OLS performance

In [12]:
from OLS import ols_method
ols_method_obj = ols_method()
W_ols_all = {}; w0_ols_all = {}; t_ols_all = {}; obj_ols_all = {}
cost_OLS_all = {}
for iter in iteration_all:
    # compute OLS performance
    W_ols_all[iter], w0_ols_all[iter], t_ols_all[iter], obj_ols_all[iter] = ols_method_obj.ols_solver("",x_train_all[iter], c_train_all[iter])
    cost_dem = (W_ols_all[iter] @ x_test_all[iter].T).T + w0_ols_all[iter]
    cost_OLS_all[iter] = perfs.compute_Oracel_Cost(arcs, grid,cost_dem,c_test_all[iter])
    # cost_OLS_all[iter] = perfs.compute_Cost_with_Prediction(arcs,w0_ols_all[iter],W_ols_all[iter], grid,c_test_all[iter],x_test_all[iter])

In [13]:
# from Shortest_Path_Model import My_ShortestPathModel
# full_shortest_model = My_ShortestPathModel()
# cost_OLS_avg_log = np.zeros(len(iteration_all))
# for iter in iteration_all:
#     # compute OLS performance
#     W_ols, w0_ols, t_ols, obj_ols = ols_method_obj.ols_solver("",x_train_all[iter], np.log(c_train_all[iter]))
#     dem_coef = np.exp((W_ols @ x_test_all[iter].T).T + w0_ols)
#     cost_OLS_log = perfs.compute_Oracel_Cost(arcs, grid,dem_coef,c_test_all[iter])
#     cost_OLS_avg_log[iter] = np.nanmean(cost_OLS_log)
# print("OLS cost = ",np.round(cost_OLS_avg_log,4))

### DDR performance

In [14]:
def obtain_DDR_out_of_sample_performance(mu_all,lamb_all,num_nodes,x_train,c_train,x_test,c_test,perfs,grid,is_log):
    lamb_index = 0
    cost_DDR = {}; w0_ddr_dict = {}; W_ddr_dict = {}
    cost_DDR_avg = np.zeros((len(mu_all),len(lamb_all)))
    mu_index = 0
    for mu in mu_all:
        lamb_index = 0
        for lamb in lamb_all:
            if is_log:
                w0_ddr_dict[mu,lamb],W_ddr_dict[mu,lamb],alpha_rst,obj_ddr = solve_DDR(arcs,lamb,mu,num_nodes,x_train,np.log(c_train))
                cost_dem = np.exp((W_ddr_dict[mu,lamb] @ x_test.T).T + w0_ddr_dict[mu,lamb])
            else:
                w0_ddr_dict[mu,lamb],W_ddr_dict[mu,lamb],alpha_rst,obj_ddr = solve_DDR(arcs,lamb,mu,num_nodes,x_train,c_train)
                cost_dem = (W_ddr_dict[mu,lamb] @ x_test.T).T + w0_ddr_dict[mu,lamb]
            cost_DDR[mu,lamb] = perfs.compute_Oracel_Cost(arcs, grid,cost_dem,c_test)
            # cost_DDR[mu,lamb] = perfs.compute_Cost_with_Prediction(arcs,w0_ddr_dict[mu,lamb],W_ddr_dict[mu,lamb], grid,c_test,x_test)
            cost_DDR_avg[mu_index,lamb_index] = np.nanmean(cost_DDR[mu,lamb])
            lamb_index = lamb_index + 1
        # print("cost_DDR_avg=",np.round(cost_DDR_avg[0,:],4))
        mu_index = mu_index + 1
    return cost_DDR,w0_ddr_dict,W_ddr_dict,cost_DDR_avg

In [15]:
def obtain_ddr_rst(iteration_all,mu_all,lamb_all,num_nodes,x_train_all,c_train_all,x_test_all,c_test_all,perfs,grid,is_log):
    cost_DDR_all = {}; w0_ddr_all = {}; W_ddr_all = {}
    cost_DDR_avg_all = {}
    for iter in iteration_all:
        cost_DDR_all[iter],w0_ddr_all[iter],W_ddr_all[iter],cost_DDR_avg_all[iter] = obtain_DDR_out_of_sample_performance(mu_all,lamb_all,num_nodes,x_train_all[iter],c_train_all[iter],x_test_all[iter],c_test_all[iter],perfs,grid,is_log)
        print("iter = ",iter,",cost_DDR_avg_all = ")
        print(np.round(cost_DDR_avg_all[iter],4))
    # print("regred_DDR_vs_OLS_all=",regred_DDR_vs_OLS_all)
    return cost_DDR_all,w0_ddr_all,W_ddr_all,cost_DDR_avg_all

In [16]:
is_log = False
mu_all = np.arange(0.1,1.0,0.1)
lamb_all = np.round(np.arange(0.0,1.0,0.1),4)
cost_DDR_all,w0_ddr_all,W_ddr_all,cost_DDR_avg_all = obtain_ddr_rst(iteration_all,mu_all,lamb_all,num_nodes,x_train_all,c_train_all,x_test_all,c_test_all,perfs,grid,is_log)

iter =  0 ,cost_DDR_avg_all = 
[[13.3374 13.3378 13.3387 13.3394 13.3408 13.3415 13.3421 13.3415 13.3433
  13.3467]
 [13.3374 13.3378 13.3367 13.3368 13.3391 13.3389 13.3386 13.3419 13.3428
  13.3435]
 [13.3374 13.3376 13.3379 13.3375 13.3371 13.3382 13.3407 13.3412 13.3364
  13.3368]
 [13.3374 13.3375 13.336  13.3365 13.3369 13.3376 13.3373 13.3359 13.3362
  13.3372]
 [13.3374 13.3378 13.337  13.3359 13.3365 13.3366 13.3376 13.337  13.3362
  13.3374]
 [13.3374 13.3374 13.3371 13.3376 13.3376 13.3368 13.3368 13.3382 13.3383
  13.339 ]
 [13.3374 13.3384 13.3377 13.3381 13.3376 13.3377 13.3385 13.3388 13.3388
  13.3385]
 [13.3374 13.3385 13.3384 13.338  13.3373 13.338  13.3379 13.3377 13.3382
  13.3386]
 [13.3374 13.3379 13.3385 13.3388 13.3391 13.3383 13.338  13.3378 13.338
  13.3381]]
iter =  1 ,cost_DDR_avg_all = 
[[13.3344 13.335  13.3361 13.3351 13.3356 13.3361 13.3365 13.339  13.3416
  13.3435]
 [13.3344 13.3348 13.3353 13.3353 13.3353 13.336  13.336  13.3361 13.3363
  13.3358]
 [1

In [17]:
regret_ddr_vs_ols_para_all = {}; regret_ddr_vs_ols_para_avg = np.zeros((len(mu_all),len(lamb_all)))
regret_ddr_vs_ols_realization_all = {}; regret_ddr_vs_ols_realization_avg = np.zeros((len(mu_all),len(lamb_all)))

cost_OLS_avg_all = np.zeros(len(iteration_all))
cost_Oracle_para_avg_all = np.zeros(len(iteration_all))
cost_Oracle_realization_avg_all = np.zeros(len(iteration_all))

for iter_index in range(len(iteration_all)):
    iter = iteration_all[iter_index]
    cost_OLS_avg_all[iter_index] = np.nanmean(cost_OLS_all[iter])
    cost_Oracle_para_avg_all[iter_index] = np.nanmean(cost_Oracle_para_all[iter])
    cost_Oracle_realization_avg_all[iter_index] = np.nanmean(cost_Oracle_realization_all[iter])

    regret_ddr_vs_ols_para_all[iter_index] = (cost_OLS_avg_all[iter_index] - cost_DDR_avg_all[iter])/np.abs(cost_OLS_avg_all[iter_index] - cost_Oracle_para_avg_all[iter_index])
    regret_ddr_vs_ols_para_avg = regret_ddr_vs_ols_para_avg + regret_ddr_vs_ols_para_all[iter_index]

    regret_ddr_vs_ols_realization_all[iter_index] = (cost_OLS_avg_all[iter_index] - cost_DDR_avg_all[iter])/np.abs(cost_OLS_avg_all[iter_index] - cost_Oracle_realization_avg_all[iter_index])
    regret_ddr_vs_ols_realization_avg = regret_ddr_vs_ols_realization_avg + regret_ddr_vs_ols_realization_all[iter_index]

regret_ddr_vs_ols_para_avg = regret_ddr_vs_ols_para_avg/len(iteration_all)
regret_ddr_vs_ols_realization_avg = regret_ddr_vs_ols_realization_avg/len(iteration_all)


In [18]:
regret_DDR_vs_OLS_para_avg_df = pd.DataFrame(regret_ddr_vs_ols_para_avg)
regret_DDR_vs_OLS_para_avg_df.index = mu_all
regret_DDR_vs_OLS_para_avg_df.columns = lamb_all
regret_DDR_vs_OLS_para_avg_df.to_csv(DataPath+"regret_DDR_vs_OLS_para_avg.csv")

regret_DDR_vs_OLS_realization_avg = pd.DataFrame(regret_ddr_vs_ols_realization_avg)
regret_DDR_vs_OLS_realization_avg.index = mu_all
regret_DDR_vs_OLS_realization_avg.columns = lamb_all
regret_DDR_vs_OLS_realization_avg.to_csv(DataPath+"regret_DDR_vs_OLS_realization_avg.csv")

In [19]:
# with open(DataPath+'cost_OLS_all.pkl', "wb") as tf:
#     pickle.dump(cost_OLS_all,tf)
# with open(DataPath+'cost_Oracle_all.pkl', "wb") as tf:
#     pickle.dump(cost_Oracle_all,tf)
# with open(DataPath+'cost_DDR_all.pkl', "wb") as tf:
#     pickle.dump(cost_DDR_all,tf)

In [20]:
# seed = 1
# lamb = 1
# cost_OLS_esti = x_test_all[seed] @ np.asarray(W_ols_all[seed]).T + w0_ols_all[seed]
# cost_DDR_esti = x_test_all[seed] @ np.asarray(W_ddr_all[seed,mu][lamb]).T + w0_ddr_all[seed,mu][lamb]

# S_test_index = 3
# print("cost_DDR_esti=",np.round(cost_DDR_esti[S_test_index,],2))
# # print("cost:(0,1)-->(1,3) = ",cost_DDR_esti[S_test_index,0] + cost_DDR_esti[S_test_index,2])
# # print("cost:(0,2)-->(2,3) = ",cost_DDR_esti[S_test_index,1] + cost_DDR_esti[S_test_index,3])

# print("cost_OLS_esti=",np.round(cost_OLS_esti[S_test_index,],2))
# # print("cost:(0,1)-->(1,3) = ",cost_OLS_esti[S_test_index,0] + cost_OLS_esti[S_test_index,2])
# # print("cost:(0,2)-->(2,3) = ",cost_OLS_esti[S_test_index,1] + cost_OLS_esti[S_test_index,3])


# print("cost_Oracle=",np.round(c_test_all[seed][S_test_index],2))
# # print("cost:(0,1)-->(1,3) = ",c_test_all[seed][S_test_index,0] + c_test_all[seed][S_test_index,2])
# # print("cost:(0,2)-->(2,3) = ",c_test_all[seed][S_test_index,1] + c_test_all[seed][S_test_index,3])

# Plot figures

In [21]:
def cross_compare2plus(c_item, c_base, c_oracle):
    c_item = np.asarray(c_item)
    c_base = np.asarray(c_base)
    c_oracle = np.asarray(c_oracle)

    N = len(c_item)
    c_diff = c_item - c_base
    lbel = np.zeros((N,1))
    
    equals = np.sum(c_diff == 0)
    wins = np.sum(c_diff < 0)
    lose = np.sum(c_diff > 0)
    
    lbel[c_diff < 0] = 1
    lbel[c_diff > 0] = -1
    
    # print("Num_train =",N,",Num_equals =",equals,",Num_wins =",wins,",Num_lose =",lose)
    # print("base cost = ", np.mean(c_base),",item cost = ",np.mean(c_item))
    if N == equals:
        win_ratio = 0.5
    else:
        win_ratio = wins/(N - equals)
    # cost_reduction = (np.mean(c_base) - np.mean(c_item) )/np.abs(np.mean(c_oracle))
    regret_reduction = (np.nanmean(c_base) - np.nanmean(c_item))/np.abs(np.nanmean(c_base) - np.nanmean(c_oracle))
    return lbel, win_ratio, regret_reduction

### DDR figure

In [22]:
# H2H_DDR_vs_OLS_all = {}; regret_reduction_DDR_vs_OLS_all = {}
# for mu in mu_all:
#     for lamb in lamb_all:
#         H2H_DDR_vs_OLS_arr = np.zeros(len(iteration_all)); regret_reduction_DDR_vs_OLS_arr = np.zeros(len(iteration_all))
#         # print("lamb = ",lamb)
#         for iter in iteration_all:
#             lbel, H2H_DDR_vs_OLS_arr[iter-1], regret_reduction_DDR_vs_OLS_arr[iter-1] = cross_compare2plus(cost_DDR_all[iter][mu,lamb],cost_OLS_all[iter], cost_Oracle_all[iter])
#         H2H_DDR_vs_OLS_all[mu,lamb] = H2H_DDR_vs_OLS_arr; regret_reduction_DDR_vs_OLS_all[mu,lamb] = regret_reduction_DDR_vs_OLS_arr

In [23]:
# mu = mu_all[0]
# lamb = lamb_all[7]
# import Figures
# file_name = DataPath + "figure_mu="+str(mu)+"_lamb="+str(lamb)
# Figures.figure_plot_upleft(H2H_DDR_vs_OLS_all[mu,lamb]*100, regret_reduction_DDR_vs_OLS_all[mu,lamb], file_name, size = (5, 5), move = [-0.12, 0.04, 0.35, 0.55], 
#                     ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

In [24]:
# import matplotlib.pyplot as plt
# fig = plt.figure()
# ax = plt.subplot()
# ax.plot(lamb_all,regret_all[seed,mu])

### SPO figure

In [25]:
# H2H_SPO_vs_OLS_arr = np.zeros(len(seed_all)); regret_reduction_SPO_vs_OLS_arr = np.zeros(len(seed_all))
# seed_index = 0
# for seed in seed_all:
#     lbel, H2H_SPO_vs_OLS_arr[seed_index], regret_reduction_SPO_vs_OLS_arr[seed_index] = cross_compare2plus(cost_EPO_all[seed]["SPO"],cost_OLS_all[seed], cost_Oracle_all[seed])
#     seed_index = seed_index + 1
# H2H_SPO_vs_OLS_all = H2H_SPO_vs_OLS_arr; regret_reduction_SPO_vs_OLS_all = regret_reduction_SPO_vs_OLS_arr

In [26]:
# Figures.figure_plot_upleft(H2H_SPO_vs_OLS_all*100, regret_reduction_SPO_vs_OLS_all, "", size = (5, 5), move = [-0.12, 0.04, 0.35, 0.55], 
#                     ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

In [27]:
# regret_reduction_SPO_vs_OLS_arr